# 量子フーリエ変換
量子フーリエ変換は既存計算機の高速フーリエ変換に対応したアルゴリズムです。高速フーリエ変換の資料はたくさんあると思いますので、今回は量子フーリエ変換QFTを見てみます。


## 量子フーリエ変換とは？
フーリエ変換は時系列の波を周波数ごとに分解します。フーリエ変換の中で離散値をとるものが離散フーリエ変換で、それを計算機上で高速処理するアルゴリズムが高速フーリエ変換です。量子フーリエ変換はこの高速フーリエ変換を量子コンピュータを使って計算を行います。

## 数理
高速フーリエ変換にとても似ています（が符号が違う気が、、、）。ある量子状態を別の量子状態に写します。

$$QFT:\mid x \rangle \mapsto \frac{1}{\sqrt{N}}\sum_{k=0}^{N-1} \omega_n^{xk}\mid k\rangle \\ \omega_n = e^{\frac{2\pi i}{N}}$$

## ビットを入力し、位相を量子状態で出力
量子フーリエ変換の入力は01のビットになります。これらのビット入力が位相の形で量子状態として出力されます。出力状態をテンソル積を用いて表現すると、

$$QFT(\mid x_1,x_2,…,x_n \rangle) = \frac{1}{\sqrt{N}}(\mid 0 \rangle + e^{2\pi i [0.x_n]} \mid 1 \rangle) \otimes … \otimes (\mid 0 \rangle + e^{2\pi i [0.x_1x_2…x_n]} \mid 1 \rangle)$$

位相にビットが現れた状態での量子状態が得られます。ここで注意したいのは、各量子状態の出現確率は確率振幅の二乗で表せられますが、どの量子状態も出現確率は同一なので測定を通じて位相が得られない点です。

## 量子回路の実装
一番簡単な回路は２量子ビットです。

<img src="./img/013_algo_qft01.png">

In [2]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
from blueqat import Circuit
import math

Circuit().x[0,1].h[0].crz(math.pi/2)[1,0].h[1].run()

array([ 5.00000000e-01+0.j , -8.32667268e-17-0.5j, -5.00000000e-01+0.j ,
        8.32667268e-17+0.5j])

つぎにN=4です。
<img src="./img/013_algo_qft02.png">

In [9]:
Circuit().x[:].h[0].crz(math.pi/2)[1,0].crz(math.pi/4)[2,0].crz(math.pi/8)[3,0].h[1].crz(math.pi/2)[2,1].crz(math.pi/4)[3,1].h[2].crz(math.pi/2)[3,2].h[3].run()

array([ 2.50000000e-01+6.93889390e-18j,  2.30969883e-01-9.56708581e-02j,
        1.76776695e-01-1.76776695e-01j,  9.56708581e-02-2.30969883e-01j,
       -5.55111512e-17-2.50000000e-01j, -9.56708581e-02-2.30969883e-01j,
       -1.76776695e-01-1.76776695e-01j, -2.30969883e-01-9.56708581e-02j,
       -2.50000000e-01-6.93889390e-18j, -2.30969883e-01+9.56708581e-02j,
       -1.76776695e-01+1.76776695e-01j, -9.56708581e-02+2.30969883e-01j,
        5.55111512e-17+2.50000000e-01j,  9.56708581e-02+2.30969883e-01j,
        1.76776695e-01+1.76776695e-01j,  2.30969883e-01+9.56708581e-02j])

このように、実際にコードを用いて簡単に行うことができました。こちらが量子フーリエ変換です。



## （応用）確かめ
量子フーリエ変換は量子状態を直接観測できないので、使いどころが難しいです。そのため最初はシミュレータで状態ベクトルを取得して、既存の高速フーリエ変換FFTと比較するのが良いでしょう。高速フーリエ変換はPythonの一般的なライブラリのNumpyに収録されていて確認ができます。

In [101]:
import numpy as np
print(np.fft.fft(np.array([0,0,0,1])/2)) 

[ 0.5+0.j   0. +0.5j -0.5+0.j   0. -0.5j]


このようにビットを入力することで、実際に量子フーリエ変換における状態ベクトルからの値と比べて確認をすることができます。上記は最初の二量子ビットの状態と対応していることが確認できます。ビットは状態ベクトルの形に直す必要があるので、1,1のビットは0001と表現されます。以上です。

# 概要

## 離散フーリエ変換
関数を三角関数の和で書き直すためにフーリエ変換というものを使います。   
具体的に言うと三角関数の和で書き直したとき、各三角関数の角振動数とその係数を取り出すということをします。

例えば sin2x なら 2, 1 を取り出すという操作です。
sinx + sin3x なら 1, 1 と 3, 1 を取り出す操作です。

このフーリエ変換は全ての点を考えるのに対し有限個の点のみで考えるフーリエ変換を離散フーリエ変換と言います。

例えば 5sin2x を考えます。これは離散フーリエ変換をすると 5, 2 が取り出せることがわかります。具体的に計算してみます。

このグラフは以下のようになります。

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F517405%2Fecd93d6d-11d6-0418-2a35-2100383acd06.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=7ae37421c295692ee627c3e12dba1a83">

これを16分割で離散フーリエ変換します。

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F517405%2Fc9fa7740-e50a-9661-979d-d9819b7776dc.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=5ddaf7e32081a85c6d1c7b04271de027">

各点の y 座標を $x_j$ と置くと

<img src="./img/013_02/013_02_0.png" width="15%">

となります。

これを離散フーリエ変換すると

<img src="./img/013_02/013_02_1.png" width="40%">

となるので $|y_k|/2$ のグラフをかくと

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F517405%2F803b4786-3773-5146-b1db-32a017ea43f2.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=e86a155a4bdd33073d1a530f1bad2416">

上のグラフは振動数 2、 振幅 5 なのできちんと表せることがわかります。

## 量子フーリエ変換
次に離散フーリエ変換から量子フーリエ変換を考えます。
上の各 $x_j$ を各ビットに埋め込むことを考えます。

すなわち入力する状態を以下のようにします。

<img src="./img/013_02/013_02_2.png" width="15%">

この $x_j$ を離散フーリエ変換すると

<img src="./img/013_02/013_02_3.png" width="20%">

となる。また、y について

<img src="./img/013_02/013_02_4.png" width="15%">

と置くと、

<img src="./img/013_02/013_02_5.png" width="30%">

括弧の部分に注目して以下の変換

<img src="./img/013_02/013_02_6.png" width="25%">

のことを量子フーリエ変換と言います。

## 回路の作成
上の式をゲートで実装するために以下のように式を変換します。

<img src="./img/013_02/013_02_7.png" width="85%">

回路は以下のようになります。

<img src="./img/013_02/013_02_8.png" width="85%">

j を量子フーリエ変換する場合は j を 2進数表示した $j=i_1...i_n$ を入力とします。
また Rn は以下の位相ゲートとします。

<img src="./img/013_02/013_02_9.png" width="15%">

以上の回路で量子フーリエ変換を実装できます。